### Kubernetes에 대해 알아보자

1. TANGO에서는 TANGO-CT라는 프로젝트 이름으로 On-premise 환경을 구성하여 docker 및 k8s 적용 사업 즉, 클라우드 전환 사업을 진행중

2. 향후 AWS를 사용할 예정

3. TANGO에 이미 On-premise 환경으로 docker와 k8s가 구축되어 있어 로컬에 VMWare 등을 설치하지 않고 업무에 사용하면서 공부

다음과 같이 구성

* Docker image 구성 및 Source build (Jenkins) 등 용도로 1식

* Master Node 1식 / Worker Node 3식

* Docker version : 19.03.12

* k8s version : 1.19.0
     

05. 쿠버네티스 설치하기 : pass

06. 쿠버네티스 시작하기부터 설명

* k8s의 기본 .. : 사실상 오늘날의 표준으로 사용되는 컨테이너 오케스트레이션 (컨테이너의 배포, 관리, 확장, 네트워킹을 자동화) 도구

0. k8s는 컨테이너 오케스트레이션 도구
    - 서버 자원 클러스터링, MSA 구조의 컨테이너 배포, 서비스 장애 복구 등 컨테이너 기반의 서비스 운영에 필요한 대부분의 오케스트레이션 기능을 폭 넓게 지원
    - 구글, 레드햇을 비롯한 많은 오픈소스 진영에서 k8s의 소스코드에 기여하고 있어 성능과 안정성 면에서 신뢰받음
    - 영속적 볼륨(Persistent Volume), 스케줄링, 장애복구, 오토 스케일링, 서비스 디스커버리 및 인그레스 등 컨테이너 기반의 클라우드를 운영할 때 필요한 대부분의 기능과 컴포넌트를 사용자가 직접 커스터마이징해서 사용할 수 있음
    - CNCF (Cloud Native Computing Foundation) 및 다른 클라우드 운영 도구들과 쉽게 연동되어 확장성이 높음 (https://www.cncf.io/)<br>
<br>
1. 모든 리소스는 Object 형태로 관리된다.
    - Pod : 컨테이너의 집합
    - Replica Set : 컨테이너의 집합을 관리하는 컨트롤러
    - Service Account : 서비스 사용자
    - Node : 노드 등 ..<br>
<br>
2. docker와 마찬가지로 kubectl이라는 명령어로 쿠버네티스를 사용한다.
3. docker와 마찬가지로 yaml 파일을 작성해서 사용할 수도 있다.
4. 쿠버네티스 노드의 역할은 크게 마스터/워커로 나뉜다.
    - Master Node : k8s가 제대로 동작할 수 있도록 클러스터를 관리하는 역할
    - Worker Node : 애플리케이션 컨테이너가 생성됨
    - Master Node에서는 API 서버(kube-apiserver), 컨트롤러 매니저(kube-controller-manager), 스케줄러(kube-scheduler), DNS 서버(core DNS) 등이 실행되고, 모든 노드에서는 오버레이 네트워크 구성을 위해 프록시(kube-proxy)와 네트워크 플러그인(Calico, flannel 등)이 실행된다.
    - 그리고 k8s 클러스터 구성을 위해 kubelet이라는 에이전트가 모든 노드에서 실행된다.
    - kubelet은 컨테이너의 생성, 삭제뿐만 아니라 마스터와 워커 노드 간의 통신 역할을 함께 담당하는 매우 중요한 에이전트이다.
    - 따라서 kubelet이 정상적으로 실행되지 않으면 해당 노드는 k8s와 제대로 연결이 되지 않을 수 있다.<br>
    <br>

* Pod : 컨테이너를 다루는 기본 단위
    - k8s에는 다양한 리소스 종류와 컴포넌트가 존재한다.
    - 컨테이너 app.을 구동하기 위해 반드시 알아야할 몇 가지 obj.가 있다.
    - pod, replica set, service, deployment<br>
    <br>
    - 1개 이상의 컨테이너로 구성된 컨테이너의 집합
    - 컨테이너 app.을 배포하기 위한 기본 단위
    - 1개의 pod 내에는 1개의 container가 존재할 수 있고 여러개의 container가 존재할 수 있다.

In [ ]:
apiVersion: v1  # yaml파일에서 정의한 object의 api 버전
kind: Pod       # 리소스의 종류
metadata:       # 라벨, 주석, 이름 등과 같은 리소스의 부가 정보
  name: nginx
spec:           # 리소스를 생성하기 위한 자세한 정보
  containers:
  - name: nginx
    image: nginx:1.14.2
    ports:
    - containerPort: 80

.. 위 기능들은 docker run으로 생성했던 컨테이너와 크게 차이가 없는데, 왜 pod라는 개념을 사용할까 ? <br>
- pod는 컨테이너 IP 주소를 갖고 있어 k8s 클러스터 내부에서 접근할 수 있음
- kubectl 명령어로 pod의 정보를 볼 수 있음
- 즉, CRI (Container Resource Interface) 제공 등의 여러 이유가 있지만 ..
- 여러 리눅스 namespace를 공유하는 여러 컨테이너들을,, 추상화된 집합으로 만들어 사용하기 위해서 ..
- pod 내 컨테이너들이 네트워크 namespace 등과 같은 리눅스 namespace를 공유해서 사용하기 때문에, 하나의 추상화된 namespace 내의 컨테이너들은 공유의 개념을 사용한다.

* Replica set
    - 동일한 pod를 여러개 만들어 관리하는 컨트롤러
    - yaml 파일에 동일한 pod를 name만 다르게 2개 만들어, kubectl apply -f 로 적용하고 강제 종료하면, 종료된 상태로만 남아있음
    - 그러면 사용자는 다시 pod를 kubectl delete -f 하고... apply -f 해야하는데.. 이러한 한계점을 해결해주는 것이 Replica set
    - Replica set의 역할
        - 정해진 수의 동일한 pod가 항상 실행되도록 관리
        - 노드 장애 등의 이유로 pod를 사용할 수 없다면 다른 노드에서 pod를 다시 생성함
    - 안정적으로 pod를 여러개 실행할 수 있고, 장애 등의 이슈가 생겨도 항상 일정한 pod의 수를 유지함
    

In [ ]:
apiVersion: apps/v1
kind: ReplicaSet            # ReplicaSet 
metadata:
  name: frontend
  labels:                   # ReplicaSet 동작원리,
    app: guestbook
    tier: frontend           
spec:
  # 케이스에 따라 레플리카를 수정한다.
  replicas: 3               # 동일한 pod를 몇 개 유지시킬 것인지 ?
  selector:
    matchLabels:            # ReplicaSet 동작 원리, 이 matchLabels를 기준으로 생성해야하는 pod를 찾음
      tier: frontend
  template:
    metadata:
      labels:
        tier: frontend      
    spec:
      containers:
      - name: php-redis
        image: gcr.io/google_samples/gb-frontend:v3

![nn](kubectl_get_pods.PNG) <br>
- 위는 StatefulSet으로 생성된 pod
- ReplicaSet은 2로 지정되어 있음

* Deployment : ReplicaSet, pod의 배포를 관리
    - 실제 k8s 운영 환경에서 ReplicaSet을 yaml 파일에서 사용하는 경우는 거의 없음
    - 대부분 Deployment라는 이름의 오브젝트를 yaml 파일에 정의해서 사용
    - 즉, 구조는 Deployment -> ReplicaSet -> pod 생성/관리 

In [ ]:
apiVersion: apps/v1
kind: Deployment            # ReplicaSet -> Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3               # 나머지는 그대로
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80

* Deployment를 사용하는 이유
    - 컨테이너 애플리케이션을 배포하고 관리하는 역할을 담당
    - 애플리케이션을 업데이트할 때 ReplicaSet의 변경사항을 저장하는 Revision을 남겨 롤백을 가능하게 하고
    - 무중단 서비스를 위한 pod의 롤링 업데이트도 가능
    - 관련 문서 : https://kubernetes.io/ko/docs/concepts/workloads/controllers/deployment/
    

* Service : pod를 연결하고 외부에 노출
    - pod의 IP는 docker 컨테이너와 마찬가지로 영속적이지 않아 항상 변한다.
    - 핵심 기능
        - 여러 개의 pod에 쉽게 접근할 수 있도록 고유한 도메인 이름을 부여
        - 여러 개의 pod에 접근할 때, 요청을 분산하는 load balancer 기능 수행
        - 클라우드 플랫폼의 load balancer, 클러스터 노드의 port 등을 통해 pod를 외부로 노출
    - 서비스의 종류
        - ClusterIP : k8s 내부에서만 pod들에 접근할 때 사용, 외부로 pod를 노출하지 않기 때문, k8s 클러스터 내부에서만 사용되는 pod에 적합
        - NodePort : pod에 접근할 수 있는 port를 클러스터의 모든 노드에 동일하게 개방, 외부에서 pod에 접근할 수 있는 서비스 타입, port는 랜덤으로 정해지지만 특정 port를 지정할 수도 있음
        - LoadBalancer : 클라우드 플랫폼에서 제공하는 load balancer를 동적으로 프로비저닝해 pod에 연결, NodePort와 마찬가지로 외부에서 pod에 접근할 수 있음, 그렇지만 일반적으로 AWS, GCP 등과 같은 클라우드 플랫폼 환경에서만 사용할 수 있음
        - ExternalName : 요청을 외부로 Redirect하는 서비스, k8s를 외부 시스템과 연동해야할 때 사용, 주로 k8s와 별개로 존재하는 legacy 시스템에 연동하는 상황에서 유용하게 사용
        

* Namespace : 리소스를 논리적으로 구분하는 장벽
    - 용도에 따라 컨테이너와 그에 관련된 리소스들을 구분지어 관리할 수 있는 일종의 논리적인 그룹<br>
    ![nn](kubectl_get_pods.PNG)<br>


* Configmap, Secret : 설정 값을 pod에 전달
    - Docker 이미지 내에 설정값 또는 설정 등을 저장하고 build 하면 변경할 수 없음
    - pod에는 yaml 파일에 환경 변수를 직접 적어 놓을 수 있음
    - 하지만 .. 하나의 application에 환경변수만 다르다면 각각 다른 두 가지 이상의 버전을 관리해야 하는 번거로움
    - 따라서 k8s에는 yaml 파일과 설정 값을 분리할 수 있는 configmap과 secret이 있음<br>
    ![nn](configmap.PNG)<br>
    <br><br>
    ![nn](configmap_yaml.PNG)<br>

* Secret
    - ssh-key, 패스워드 등의 민감 정보를 저장하기 위한 용도로 사용
    - Secret에 값을 저장할 때, k8s가 기본적으로 base64로 인코딩하여 처리
    - 사용 방법은 configmap과 동일
        - volume으로 mount해서 사용하거나
        - 파일을 직접 마운트 하거나
        - 환경변수로 사용하거나
        - key/value로 사용하거나 등등..
        